<h1> ______________________________________ </h1>

<h1> Fiber panels with Kr38m decays simulation analysis </h1>
<h1> ______________________________________ </h1>

<p style="font-size: 17px; color: black;"> In this Notebook we analyse the results from the 10000 MC Kr38m decay events simulated in a "realistic" fiber barrel to check if we're able to see the krypton signal with enough resolution. </p>




<h1> ____________ </h1>
<h2> This version </h2>
<h1> ____________ </h1>

-  <p style="font-size: 17px; color: black;"> 10K events. </p>

-  <p style="font-size: 17px; color: black;"> Geant4 fundamental units: </p>

 -  <p style="font-size: 17px; color: black;"> Length [L]: mm (milimeter) </p>
 -  <p style="font-size: 17px; color: black;"> Time [T]: ns (nanosecond) </p>
 -  <p style="font-size: 17px; color: black;"> Energy [E]: MeV (megaelectronvolt) </p>





In [223]:
import numpy              as np
import pandas             as pd

import scipy              
from scipy                import stats, integrate
from scipy.signal         import convolve

import matplotlib.pyplot  as plt

from matplotlib.ticker    import FormatStrFormatter # to set the format of the plot's axis
from matplotlib.patches   import Rectangle # To add blanck spaces in tabular legends

import os
import math
import tables             as tb

<h1> _________________________________________________________________________________________________________ </h1>


<h1> __________________________ </h1>
<h2> Functions </h2>

-  <p style="font-size: 17px; color: black;"> <b> gauss(x, a,  mu, sigma): </b> Gaussian normalized to $a$. </p>


-  <p style="font-size: 17px; color: black;"> <b> gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1): </b> Sum of 2 gaussians. </p>


-  <p style="font-size: 17px; color: black;"> <b> dirac(x, x0): </b> Dirac delta at $x0$. </p>


-  <p style="font-size: 17px; color: black;"> <b> EL_yield(E, p, d): </b> It gives you the EL gain in $[photons/e^-]$ as a function of the electric field $E[kV/cm]$, pressure $p[bar]$ and the average absoption length $d[cm]$. We end up not using this formula cause we use directly a fixed value of EL gain. </p>




In [224]:
# Fitting distributions to define

# note: pdf are normilized to 1 so we have yo add an amplitude param
def gauss(x, a,  mu, sigma):
    
    gaussian = stats.norm.pdf(np.sort(x), mu, sigma) 
    
    return (a/gaussian.sum())*gaussian

def gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1):
    return a0*stats.norm.pdf(x, mu0, sigma0)  + a1*stats.norm.pdf(x, mu1, sigma1)

def dirac(x, x0):
    return np.where((x - x0) == 0, 1, 0)

def EL_yield(E, p, d):
    # E[kV/cm] electric field 
    # p[bar] pressure
    # d[cm] average absorption depth
    return (140*E/p - 116)*d*p # [photons/e⁻]


<h1> __________________________ </h1>
<h2> Global parameters </h2>

-  <p style="font-size: 17px; color: black;"> <b> Pandas dataset params: </b> Allows to set the max number of columns and rows that are shown in a pandas dataset. </p>

In [225]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)

-  <p style="font-size: 17px; color: black;"> <b> Plots params: </b> In case you want to fix the parameters for all plots. </p>

In [226]:
# plt.rcParams["figure.figsize"] = 10, 8
# plt.rcParams["font.size"] = 10

-  <p style="font-size: 17px; color: black;"> <b> Analysis parameters </b> </p>

 -  <p style="font-size: 17px; color: black;"> <b> patata: </b> patata. </p>
 

<h1> __________________________ </h1>
<h2> Data </h2>

-  <p style="font-size: 17px; color: black;"> <b> Reading the file </b> </p>

-  <p style="font-size: 17px; color: black;"> File's path </p>

In [227]:
# path = '/home/investigator/mariandbt/python/data/'
# path = path + '/20230824_fib_pan_kr'

path = '/home/investigator/mariandbt/nexus'

filename = os.path.join(path, "Fib_pan_meth.next.h5")

-  <p style="font-size: 17px; color: black;"> In a .h5 file there are several objects, so we print them all to then pick the one we're interested in </p>

In [228]:
with tb.open_file(filename) as file:
    print(file)

/home/investigator/mariandbt/nexus/Fib_pan_meth.next.h5 (File) ''
Last modif.: '2023-08-28T10:24:44+00:00'
Object Tree: 
/ (RootGroup) ''
/MC (Group) ''
/MC/configuration (Table(43,)) ''
/MC/hits (Table(0,)) ''
/MC/particles (Table(24099,)) ''
/MC/sns_positions (Table(89,)) ''
/MC/sns_response (Table(297,)) ''



-  <p style="font-size: 17px; color: black;"> Read the file and make a copy to work with, this way it's easier to re-start the copy instead of re-reading the file. </p>

In [229]:
# %%timeit # WITH THIS YOU'RE ONLY TIMING, BUT IT DOESN'T REALLY RUN THE COMMANDS YOU TIME
# data = pd.read_hdf(filename, "/MC/particles")
# data = pd.read_hdf(filename, "/MC/sns_positions")
data = pd.read_hdf(filename, "/MC/sns_response")
# data = pd.read_hdf(filename, "/MC/hits")
# dst = pd.read_hdf(filename, "/MC/hits").groupby("event_id").get_group(event_id)

In [230]:
dst = data.copy()

In [231]:
# dst.final_volume.unique() 
# dst.initial_volume.unique()
# dst.particle_name.unique()
dst.sensor_id.unique(), len(dst.sensor_id.unique())

(array([3085, 3092, 3087, 3127, 3081, 3115, 3090, 3080, 3156, 3091, 3099,
        3104, 3100, 3135, 3164, 3088, 3130, 3079, 3120, 3154, 3096, 3143,
        3132, 3107, 3121, 3125, 3117, 3095, 3124, 3084, 3136, 3166, 3142,
        3110, 3106, 3129, 3122, 3113, 3119, 3131, 3155, 3108, 3158, 3114,
        3128, 3094, 3151, 3109, 3134, 3078, 3102, 3137, 3101, 3093, 3082,
        3157, 3123, 3148, 3118, 3162, 3098, 3105, 3147, 3145, 3140, 3160,
        3165, 3163, 3086, 3112, 3146, 3133, 3097, 3126, 3167, 3144, 3153,
        3089, 3116, 3139, 3103, 3083, 3152, 3141, 3138, 3149, 3150, 3161,
        3111], dtype=uint32),
 89)

In [240]:
dst.time_bin.unique(), len(dst.time_bin.unique())

(array([ 2, 10,  6,  8,  7, 32, 37, 13, 33, 17, 12,  3, 24, 28, 44,  4, 11,
        30, 21, 15,  5, 22,  9,  1, 27, 18, 20, 16, 25, 42, 76, 34, 23, 47,
        48, 31, 50, 14, 29, 26, 19, 60, 35, 39, 43], dtype=uint64),
 45)

In [234]:
# dst.loc[dst.final_volume.str.contains('SENS')]
# dst.loc[(dst.initial_volume != 'WORLD') & (dst.primary == 0)]
dst

,event_id,sensor_id,time_bin,charge
0,0,3085,2,1
1,0,3085,10,1
2,0,3092,6,1
3,0,3092,8,1
4,0,3087,7,1
5,0,3127,32,1
6,0,3081,37,1
7,0,3115,2,1
8,0,3115,13,1
9,0,3115,33,1


-  <p style="font-size: 17px; color: black;"> <b> Tests on the data: </b> We perform some tests on the dataset to check the simulation is valid. </p>

-  <p style="font-size: 17px; color: black;"> Group the data by sensors. </p>

In [235]:
# g = dst.groupby(dst.sensor_id)
g = dst.groupby(dst.event_id)


In [236]:
g.charge.sum()

event_id
0    28
1    40
2    36
3    29
4    28
5    24
6    35
7    30
8    28
9    26
Name: charge, dtype: uint32

<h1> __________________________ </h1>
<h2> Analysis </h2>

<h2> Charge distribution in an event. </h2>

-  <p style="font-size: 17px; color: black;"> One event </p>

In [237]:
ev = 5

In [238]:
wvf = dst.groupby(dst.event_id).get_group(ev)

In [239]:
# wvf.groupby(wvf.sensor_id).get_group(3163)
wvf

,event_id,sensor_id,time_bin,charge
156,5,3167,50,1
157,5,3146,2,1
158,5,3127,24,2
159,5,3117,7,1
160,5,3084,4,1
161,5,3160,4,1
162,5,3160,11,1
163,5,3134,12,1
164,5,3109,2,1
165,5,3102,14,1


In [ ]:
plt.hist(wvf.groupby(wvf.))